In [74]:
import pandas as pd
from collections import Counter

def map_banda(banda):
    banda_order = ["BUENA", "RAZONABLEMENTE BUENA", "REGULAR", "DESFAVORABLE", "MUY DESFAVORABLE", "EXTREMADAMENTE DESFAVORABLE"]
    try:
        return banda_order.index(banda)
    except ValueError:
        # generación de mensaje de error en caso de no encontrar el valor de banda
        return "Error: Valor de banda no encontrado"

# Definir una función para leer y procesar cada sheet
def process_sheet(sheet_name):
    #Leer los datos de la sheet
    df = pd.read_excel('resultados_INCA_MAYO23.xlsx', sheet_name=sheet_name)
    
    # Creación de un nuevo Dataframe con la estructura deseada
    df_area = pd.DataFrame(columns=['Datetime', 'Hour', 'Area', 'Banda', 'Agente en banda superior'])
    
    # Iteración sobre cada hora
    for hour in df['Hour'].unique():
        # Filtrar el dataframe según la hora
        df_hour = df[df['Hour'] == hour]

        # Inicialización de valor por defecto a la variable "area"
        area = 'Other'
        
        # Creación de un nuevo Dataframe con la hora actual
        df_area_hour = pd.DataFrame(columns=['Datetime', 'Hour', 'Area', 'Banda', 'Agente en banda superior'])

        # Comprobar que está 'K-A3 APTICA 1' y 'K-A3 APTICA 3'
        if any(device.strip() in ['K-A3 APTICA 1', 'K-A3 APTICA 3'] for device in df_hour['Dispositivo'].values):
            # print("Area1 condition met")
            area = 'Area1'
            agente = max(df_hour[df_hour['Dispositivo'] == 'K-A3 APTICA 1']['Agente en banda superior'].max(),
                         df_hour[df_hour['Dispositivo'] == 'K-A3 APTICA 3']['Agente en banda superior'].max(),
                         key=map_banda)
            banda = max(df_hour[df_hour['Dispositivo'] == 'K-A3 APTICA 1']['Banda'].max(),
                        df_hour[df_hour['Dispositivo'] == 'K-A3 APTICA 3']['Banda'].max(),
                        key=map_banda)
            # Añadir los valores al nuevo DataFrame para la hora actual
            df_area_hour = pd.concat([df_area_hour, pd.DataFrame({'Datetime': [df_hour['Datetime'].iloc[0]],
                                                                  'Hour': [hour],
                                                                  'Area': [area],
                                                                  'Banda': [banda],
                                                                  'Agente en banda superior': [agente]})], ignore_index=True)

        # Comprobar que está 'K-A3 APTICA 2' a pesar de la condición anterior
        if any(device.strip() == 'K-A3 APTICA 2' for device in df_hour['Dispositivo'].values):
            # print(f"Area2 condition met")
            area = 'Area2'
            device = 'K-A3 APTICA 2'  # We already know this is the device
            agente = df_hour[df_hour['Dispositivo'].str.strip() == device.strip()]['Agente en banda superior'].max()
            banda = df_hour[df_hour['Dispositivo'].str.strip() == device.strip()]['Banda'].max()
            # Añadir los valores al nuevo DataFrame para la hora actual
            df_area_hour = pd.concat([df_area_hour, pd.DataFrame({'Datetime': [df_hour['Datetime'].iloc[0]],
                                                                  'Hour': [hour],
                                                                  'Area': [area],
                                                                  'Banda': [banda],
                                                                  'Agente en banda superior': [agente]})], ignore_index=True)

        # Comprobar que están 'K-A3 APTICA 4, 5, 6, 7' a pesar de la condición anterior
          # Check for 'K-A3 APTICA 4, 5, 6, 7' regardless of the previous condition
        if any(device.strip() in ['K-A3 APTICA 4', 'K-A3 APTICA 5', 'K-A3 APTICA 6', 'K-A3 APTICA 7'] for device in df_hour['Dispositivo'].values):
            # print(f"Area3 condition met")
            area = 'Area3'
            bandas = df_hour[df_hour['Dispositivo'].isin(['K-A3 APTICA 4', 'K-A3 APTICA 5', 'K-A3 APTICA 6', 'K-A3 APTICA 7'])]['Banda'].values
            counter = Counter(bandas)
            most_common_banda = counter.most_common(1)[0][0]

            if most_common_banda in ["MUY DESFAVORABLE", "EXTREMADAMENTE DESFAVORABLE"]:
                banda = most_common_banda
                agente = df_hour[df_hour['Banda'] == banda]['Agente en banda superior'].max()
            else:
                banda = max(df_hour[df_hour['Dispositivo'] == 'K-A3 APTICA 4']['Banda'].max(),
                            df_hour[df_hour['Dispositivo'] == 'K-A3 APTICA 5']['Banda'].max(),
                            df_hour[df_hour['Dispositivo'] == 'K-A3 APTICA 6']['Banda'].max(),
                            df_hour[df_hour['Dispositivo'] == 'K-A3 APTICA 7']['Banda'].max(),
                            key=map_banda)
                agente = max(df_hour[df_hour['Dispositivo'] == 'K-A3 APTICA 4']['Agente en banda superior'].max(),
                             df_hour[df_hour['Dispositivo'] == 'K-A3 APTICA 5']['Agente en banda superior'].max(),
                             df_hour[df_hour['Dispositivo'] == 'K-A3 APTICA 6']['Agente en banda superior'].max(),
                             df_hour[df_hour['Dispositivo'] == 'K-A3 APTICA 7']['Agente en banda superior'].max(),
                             key=map_banda)

            # Append the values to the new DataFrame for the current hour
            df_area_hour = pd.concat([df_area_hour, pd.DataFrame({'Datetime': [df_hour['Datetime'].iloc[0]],
                                                                  'Hour': [hour],
                                                                  'Area': [area],
                                                                  'Banda': [banda],
                                                                  'Agente en banda superior': [agente]})], ignore_index=True)

       
        # Si ninguna condición se cumple, dar valor al Area de "OTRA"
        if area == 'Otra':
            # print("Other condition met")
            area = df_hour['Dispositivo'].values[0]
            agente = df_hour['Agente en banda superior'].max()
            banda = df_hour['Banda'].max()
            # Añadir los valores al nuevo DataFrame para la hora actual
            df_area_hour = pd.concat([df_area_hour, pd.DataFrame({'Datetime': [df_hour['Datetime'].iloc[0]],
                                                                  'Hour': [hour],
                                                                  'Area': [area],
                                                                  'Banda': [banda],
                                                                  'Agente en banda superior': [agente]})], ignore_index=True)

        # Añadir el Dataframe de la hora actual al dataframe general
        df_area = pd.concat([df_area, df_area_hour], ignore_index=True)

    return df_area

# Crear un archivo nuevo Excel para guardar los datos agrupados
output_file = 'tiriri_result_MAYO.xlsx'
with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    # Iterar sobre cada hoja del archivo excel original
    for sheet_name in pd.ExcelFile('resultados_INCA_MAYO23.xlsx').sheet_names:
        # Procesar la hoja y escribir los resultados en el nuevo archivo
        processed_data = process_sheet(sheet_name)
        processed_data.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Result saved to {output_file}")


Result saved to tiriri_result_MAYO.xlsx
